In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import openslide
from probreg import cpd
from probreg import transformation as tf
import cv2
from PIL import Image
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json
import wandb

In [3]:
import sys
sys.path.append("..")
from registration_tree import Rect, QuadTree

In [4]:
sweep_config = {
    'method': 'random', #'bayes' # 'random'
}

metric = {
    'name': 'dist_mean',
    'goal': 'minimise'   
    }


sweep_config['metric'] = metric

In [5]:
parameters_dict = {
    'method': {
        'values': ['orb', "sift"]
        },
    'maxFeatures': {
        #'values': [64, 128, 256, 512, 768, 1024, 2048]
        'values': [512, 768, 1024, 2048]
        },
    'crossCheck': {
        'values': [False]
        },
    'flann': {
        'values': [False, True]
        },
    'ratio': {
        #'values': [.1, .2, .3, .4, .5, .6, .7, .8, .9]
        'values': [.4, .5, .6, .7]
        },
    'use_gray': {
        'values': [True, False]
        },
    'target_depth': {
        'values': [0]
        },
    'thumbnail_size': {
        #'values': [(128, 128), (256, 256), (512, 512), (768, 768), (1024, 1024), (2048, 2048), (4096, 4096), (8192, 8192)]
        'values': [(1024, 1024), (2048, 2048), (4096, 4096), (8192, 8192)]
        },    
    
    'image_type': {
        'values': ["Cyto"]
        },
    
    'source_scanner': {
        'values': ["Aperio"]
        },
}
sweep_config['parameters'] = parameters_dict

In [6]:
sweep_id = wandb.sweep(sweep_config, project="quadtree")
sweep_id

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable


Create sweep with ID: hy4huvzx
Sweep URL: https://wandb.ai/christianml/quadtree/sweeps/hy4huvzx


'hy4huvzx'

In [7]:
def add_help_fields(frame):
    
    frame["image_name_stem"] = [Path(image_name).stem for image_name in frame["image_name"]]    
    frame["patient_id"] = [name.split("_")[2] for name in frame["image_name"]]

    frame["x1"] = [json.loads(vector.replace("\'","\""))['x1'] for vector in frame["vector"]]
    frame["y1"] = [json.loads(vector.replace("\'","\""))['y1'] for vector in frame["vector"]]

    frame["x2"] = [json.loads(vector.replace("\'","\""))['x2'] for vector in frame["vector"]]
    frame["y2"] = [json.loads(vector.replace("\'","\""))['y2'] for vector in frame["vector"]]

    frame["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(frame["x1"], frame["x2"])]
    frame["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(frame["y1"], frame["y2"])]
    
    frame["center"] = [np.array((center_x, center_y)) for center_x, center_y in zip(frame["center_x"], frame["center_y"])]

    frame["anno_width"] = [x2-x1 for x1, x2 in zip(frame["x1"], frame["x2"])]
    frame["anno_height"]= [y2-y1 for y1, y2 in zip(frame["y1"], frame["y2"])]
    
    return frame

In [8]:
folder = Path("..")

slide_files = {path.name: path for path in Path("D:/Datasets/ScannerStudy").glob("*/*/*.*")}

In [9]:
def train(config=None):
    
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        
        annotations = add_help_fields(pd.read_csv(folder / "Validation/GT.csv"))
        annotations = annotations[annotations["image_type"] == config.image_type]
        
        source_scanner_annotations = annotations[annotations["scanner"] == config.source_scanner]
    
        dist_list, q_list, sigma2_list = [], [], []
                                  
        for patient_id in tqdm(source_scanner_annotations["patient_id"].unique()):

            source_annos = source_scanner_annotations[source_scanner_annotations["patient_id"] == patient_id]
            source_anno = source_annos.iloc[0]

            target_patient_annotations = annotations[annotations["patient_id"] == patient_id]

            for target_image_name in tqdm(target_patient_annotations["image_name"].unique()):

                target_annos = target_patient_annotations[target_patient_annotations["image_name"] == target_image_name]
                target_anno = target_annos.iloc[0]

                source_slide = openslide.OpenSlide(str(slide_files[source_anno.image_name]))
                target_slide = openslide.OpenSlide(str(slide_files[target_anno.image_name]))

                source_dimension = Rect.create(Rect, 0, 0, source_slide.dimensions[0], source_slide.dimensions[1])
                target_dimension = Rect.create(Rect, 0, 0, target_slide.dimensions[0], target_slide.dimensions[1])


                qtree = QuadTree(source_dimension, source_slide, target_dimension, target_slide, **config)
                                  
                q_list.append(qtree.q)
                sigma2_list.append(qtree.sigma2)
                
                for type_name in source_annos["type_name"].unique():

                    source_anno = source_annos[source_annos["type_name"] == type_name].iloc[0]
                    target_anno = target_annos[target_annos["type_name"] == type_name].iloc[0]

                    box = [source_anno.center_x, source_anno.center_y, source_anno.anno_width, source_anno.anno_height]
                    target_box = [target_anno.center_x, target_anno.center_y, target_anno.anno_width, target_anno.anno_height]

                    trans_box = qtree.transform_boxes(np.array([box]))[0]

                    distance = np.linalg.norm(target_box[:2]-trans_box[:2])

                    dist_list.append(distance)
        
        dist_list, q_list, sigma2_list = np.array(dist_list), np.array(q_list), np.array(sigma2_list)
        
        wandb.log({
            "dist_mean": dist_list.mean(),
            "dist_min": dist_list.min(),
            "dist_max": dist_list.max(),
            
            "q_mean": q_list.mean(),
            "q_min": q_list.min(),
            "q_max": q_list.max(),
            
            "sigma2_mean": sigma2_list.mean(),
            "sigma2_min": sigma2_list.min(),
            "sigma2_max": sigma2_list.max(),
        })

In [10]:
sweep_id

'hy4huvzx'

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 1pevcbpp with config:
wandb: 	crossCheck: False
wandb: 	flann: False
wandb: 	image_type: Cyto
wandb: 	maxFeatures: 768
wandb: 	method: sift
wandb: 	ratio: 0.4
wandb: 	source_scanner: Aperio
wandb: 	target_depth: 0
wandb: 	thumbnail_size: [4096, 4096]
wandb: 	use_gray: True
Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: christianml (use `wandb login --relogin` to force relogin)


 60%|██████████████████████████████████████████████████▍                                 | 3/5 [04:02<02:41, 80.99s/it]


Run 1pevcbpp errored: ValueError('Expected n_neighbors > 0. Got 0',)
wandb: ERROR Run 1pevcbpp errored: ValueError('Expected n_neighbors > 0. Got 0',)
wandb: Agent Starting Run: 4utuc009 with config:
wandb: 	crossCheck: False
wandb: 	flann: True
wandb: 	image_type: Cyto
wandb: 	maxFeatures: 1024
wandb: 	method: sift
wandb: 	ratio: 0.5
wandb: 	source_scanner: Aperio
wandb: 	target_depth: 0
wandb: 	thumbnail_size: [8192, 8192]
wandb: 	use_gray: False
Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [36:50<00:00, 442.03s/it]


dist_mean,14.52075
dist_min,0.0
dist_max,466.92616
q_mean,-3873.19111
q_min,-16517.00195
q_max,586.96357
sigma2_mean,12.51853
sigma2_min,0.0
sigma2_max,26.71781
_step,0
_runtime,2214


...

Run iyed0phq errored: MemoryError()
wandb: ERROR Run iyed0phq errored: MemoryError()
wandb: Agent Starting Run: j7arw4ji with config:
wandb: 	crossCheck: False
wandb: 	flann: True
wandb: 	image_type: Cyto
wandb: 	maxFeatures: 1024
wandb: 	method: orb
wandb: 	ratio: 0.4
wandb: 	source_scanner: Aperio
wandb: 	target_depth: 0
wandb: 	thumbnail_size: [1024, 1024]
wandb: 	use_gray: True
Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]


Run j7arw4ji errored: MemoryError()
wandb: ERROR Run j7arw4ji errored: MemoryError()
wandb: Agent Starting Run: cbk7cha5 with config:
wandb: 	crossCheck: False
wandb: 	flann: False
wandb: 	image_type: Cyto
wandb: 	maxFeatures: 512
wandb: 	method: orb
wandb: 	ratio: 0.7
wandb: 	source_scanner: Aperio
wandb: 	target_depth: 0
wandb: 	thumbnail_size: [8192, 8192]
wandb: 	use_gray: False
Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]


Run cbk7cha5 errored: MemoryError()
wandb: ERROR Run cbk7cha5 errored: MemoryError()
wandb: Agent Starting Run: o79fl7ub with config:
wandb: 	crossCheck: False
wandb: 	flann: True
wandb: 	image_type: Cyto
wandb: 	maxFeatures: 2048
wandb: 	method: orb
wandb: 	ratio: 0.5
wandb: 	source_scanner: Aperio
wandb: 	target_depth: 0
wandb: 	thumbnail_size: [2048, 2048]
wandb: 	use_gray: False
Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
